# Notebook 3 — Simulator vs Hardware (IBM Runtime)
**Goal:** Run the same circuit on a simulator and on real IBM hardware (if available).

> Hardware access requires an IBM Quantum account and API token. See `00_Environment_Setup.md`.

In [ ]:
# Circuit: simple Bell state for hardware comparison
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0,1], [0,1])
qc.draw('mpl')

In [ ]:
# --- Run on simulator
backend_sim = Aer.get_backend('qasm_simulator')
sim_result = execute(qc, backend_sim, shots=2048).result()
sim_counts = sim_result.get_counts()
print("Simulator counts:", sim_counts)
plot_histogram(sim_counts)

In [ ]:
# --- Try IBM Runtime (modern API). If unavailable, this cell will print a helpful message.
try:
    from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

    service = QiskitRuntimeService()  # requires saved account
    sampler = Sampler()
    job = sampler.run(qc, shots=2048)
    hw_result = job.result()
    hw_counts = hw_result.quasi_dists[0] if hasattr(hw_result, "quasi_dists") else hw_result
    print("Hardware (sampler) quasi-dists or counts:", hw_counts)
except Exception as e:
    print("IBM Runtime path not available in this environment:", e)

In [ ]:
# --- Legacy IBMQ path (may be deprecated). Useful if your env still supports it.
try:
    from qiskit import IBMQ
    IBMQ.load_account()
    provider = IBMQ.get_provider(hub='ibm-q')
    backend = provider.backends(filters=lambda b: b.configuration().n_qubits >= 2 and not b.configuration().simulator)[0]
    print("Selected backend:", backend)
    job = execute(qc, backend=backend, shots=2048)
    print("Job ID:", job.job_id())
    result = job.result()
    hw_counts_legacy = result.get_counts()
    print("Hardware counts (legacy):", hw_counts_legacy)
except Exception as e:
    print("Legacy IBMQ path not available:", e)

**Discussion**
- Compare simulator vs hardware results; notice noise on hardware.
- Talk about error sources and simple mitigation strategies.